#Data Cleaning Actions (Python)

In [2]:
import pandas as pd
import math

In [1]:
!git clone https://github.com/hhlim2/cs513_team6.git

Cloning into 'cs513_team6'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (7/7), 7.63 MiB | 10.96 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [3]:
# upload data-cleaning-dish.csv file in files tab

dish_df = pd.read_csv('/content/cs513_team6/Dish-Jul15.csv')
dish_df.head()

,id,name,name_cluster_coffee,name_cluster_contains_coffee,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,lowest_price_missing,highest_price,highest_price_missing
0,96,Coffee,coffee,1,7740,8484,1,2928,0.0,False,30.0,False
1,118,Large pot of coffee,coffee (pot),1,5,5,1900,1963,0.0,False,0.2,False
2,119,Cup of coffee (served in small pot),coffee,1,1,1,1900,1900,0.0,False,0.0,False
3,174,Demi-tasse,espresso,1,851,878,1892,1979,0.0,False,15.0,False
4,321,Tea and Coffee,coffee,1,42,42,1900,1937,0.1,False,0.1,False


## Enforce menus_appeared Integrity Constraints

1.   Data type constraint integer must be met
2. Value cannot be negative
3. Value must be at least 1
4. Value cannot be more than 17545 (no. of records in Menu table, counted by Excel)


In [4]:
dish_df['menus_appeared'].describe()

count    6110.000000
mean        4.929460
std       102.689166
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max      7740.000000
Name: menus_appeared, dtype: float64

In [5]:
dish_df.loc[dish_df['menus_appeared'] == 0, 'menus_appeared'] = 1

In [6]:
dish_df['menus_appeared'].describe()

count    6110.000000
mean        4.936007
std       102.688883
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max      7740.000000
Name: menus_appeared, dtype: float64

## Enforce first_appeared Integrity Constraints
1. Data type constraint year of format YYYY must be met i.e. each Y is a digit
2. Value must start with either 1 or 2 as the first digit
3. Value must be at least 1840 (based on narrative description of dataset on NYPL menus website)
4. Value cannot be greater than 2021 (the year in which the dataset D was uploaded to Box, which we assume to be the year it was downloaded from NYPL)
5. Value must be smaller or equal to last_appeared in the same record



In [7]:
dish_df['first_appeared'].describe()

count    6110.000000
mean     1656.781997
std       679.643346
min         0.000000
25%      1900.000000
50%      1931.000000
75%      1953.000000
max      2012.000000
Name: first_appeared, dtype: float64

In [17]:
#fix first_appeared data
first_appeared_fixed = dish_df['first_appeared'].values.tolist()

for i in range(len(dish_df)):
  if first_appeared_fixed[i] < 1840:
      first_appeared_fixed[i] = 1840
  if first_appeared_fixed[i] > 2021:
      first_appeared_fixed[i] = dish_df['last_appeared'][i]

# print(len(first_appeared_fixed))

# print(sorted(first_appeared_fixed))

#save fixed data to dataframe
dish_df['first_appeared_fixed'] = first_appeared_fixed

In [18]:
dish_df['first_appeared_fixed'].describe()

count    6110.000000
mean     1921.488216
std        41.356094
min      1840.000000
25%      1900.000000
50%      1931.000000
75%      1953.000000
max      2012.000000
Name: first_appeared_fixed, dtype: float64

## Enforce last_appeared Integrity Constraints
1. Data type constraint year of format YYYY must be met i.e. each Y is a digit
2. Value must start with either 1 or 2 as the first digit
3. Value must be at least 1840 (based on narrative description of dataset on NYPL menus website)
4. Value cannot be greater than 2021 (the year in which the dataset D was uploaded to Box, which we assume to be the year it was downloaded from NYPL)
5. Value must be smaller or equal to last_appeared in the same record

In [10]:
dish_df['last_appeared'].describe()

count    6110.000000
mean     1661.134206
std       679.286385
min         0.000000
25%      1905.000000
50%      1933.000000
75%      1957.000000
max      2928.000000
Name: last_appeared, dtype: float64

In [22]:
#fix last_appeared data
last_appeared_fixed = dish_df['last_appeared'].values.tolist()

for i in range(len(dish_df)):
  if last_appeared_fixed[i] < 1840:
    last_appeared_fixed[i] = first_appeared_fixed[i]
  if last_appeared_fixed[i] > 2021:
    last_appeared_fixed[i] = 2021


# print(len(last_appeared_fixed))

# print(sorted(last_appeared_fixed))
# print(last_appeared_fixed)

#save fixed data to dataframe
dish_df['last_appeared_fixed'] = last_appeared_fixed

In [23]:
dish_df['last_appeared_fixed'].describe()

count    6110.000000
mean     1924.203437
std        42.194403
min      1840.000000
25%      1905.000000
50%      1933.000000
75%      1957.000000
max      2021.000000
Name: last_appeared_fixed, dtype: float64

In [24]:
# check that all first appeared years are less than or equal to last appeared years
(dish_df['first_appeared_fixed'] > dish_df['last_appeared_fixed']).value_counts()


False    6110
Name: count, dtype: int64

## Enforce lowest_price Integrity Constraints

1. Data type constraint float must be met
2. Value cannot be negative or 0
3. Value must be smaller or equal to highest_price in the same record


In [25]:
dish_df['lowest_price'].describe()

count    6110.000000
mean        0.443692
std         2.548177
min         0.000000
25%         0.000000
50%         0.000000
75%         0.250000
max       150.000000
Name: lowest_price, dtype: float64

## Enforce highest_price Integrity Constraints

1. Data type constraint float must be met
2. Value cannot be negative or 0
3. Value must be larger or equal to lowest_price in the same record


In [26]:
dish_df['highest_price'].describe()

count    6110.000000
mean        0.886643
std        13.294788
min         0.000000
25%         0.000000
50%         0.050000
75%         0.347500
max      1000.000000
Name: highest_price, dtype: float64

In [27]:
(dish_df['lowest_price'] <= dish_df['highest_price']).value_counts()

True    6110
Name: count, dtype: int64

In [28]:
#Save final cleaned dataset
dish_df.to_csv('/content/Data-Cleaning-Dish_Final_15Jul.csv')